In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder
%matplotlib inline

In [41]:
df=pd.read_csv('../../data/titanic_train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
# Works out Embarked_encoded based on p(Survival|Embarked='x')
encoder = TargetEncoder()
df['Embarked_encoded'] = encoder.fit_transform(df['Embarked'], df['Survived'])

C:\DataScience\Anaconda\envs\p37\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [43]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_encoded
43,44,1,2,"Laroche, Miss. Simonne Marie Anne Andree",female,3.0,1,2,SC/Paris 2123,41.5792,NaN,C,0.553571
529,530,0,2,"Hocking, Mr. Richard George",male,23.0,2,1,29104,11.5000,NaN,S,0.336957
525,526,0,3,"Farrell, Mr. James",male,40.5,0,0,367232,7.7500,NaN,Q,0.389610
237,238,1,2,"Collyer, Miss. Marjorie ""Lottie""",female,8.0,0,2,C.A. 31921,26.2500,NaN,S,0.336957
326,327,0,3,"Nysveen, Mr. Johan Hansen",male,61.0,0,0,345364,6.2375,NaN,S,0.336957


In [44]:

print(df[df['Embarked'] == 'S'].count())
print(df.groupby(['Embarked','Survived'])['Survived'].count())
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

df['Age'] = df[['Age','Pclass']].apply(impute_age,axis=1)
print(df.shape)

PassengerId         644
Survived            644
Pclass              644
Name                644
Sex                 644
Age                 554
SibSp               644
Parch               644
Ticket              644
Fare                644
Cabin               129
Embarked            644
Embarked_encoded    644
dtype: int64
Embarked  Survived
C         0            75
          1            93
Q         0            47
          1            30
S         0           427
          1           217
Name: Survived, dtype: int64
(891, 13)


In [45]:
df.drop(['Name', 'Ticket', 'Cabin', 'Embarked', 'Sex'], axis=1, inplace=True)
df.sample(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_encoded
464,465,0,3,24.0,0,0,8.0500,0.336957
33,34,0,2,66.0,0,0,10.5000,0.336957
171,172,0,3,4.0,4,1,29.1250,0.389610
362,363,0,3,45.0,0,1,14.4542,0.553571
726,727,1,2,30.0,3,0,21.0000,0.336957


In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived',axis=1), 
                                                    df['Survived'], test_size=0.30, random_state=101)

In [47]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression( max_iter=5000)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)   

In [48]:
import statsmodels.api as sm

# Function to calculate VIF
def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        #print(r_squared)
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

X=df.drop(['Survived'],axis=1)
calculate_vif(X)

,Var,Vif
6,Embarked_encoded,13.11
1,Pclass,7.84
2,Age,5.24
0,PassengerId,3.79
5,Fare,2.21
3,SibSp,1.56
4,Parch,1.55


In [53]:
from sklearn import metrics
metrics.confusion_matrix(y_test, predictions)

array([[135,  19],
       [ 62,  52]], dtype=int64)

In [59]:
print('accuracy = {:7.4f}'.format(logmodel.score(X_test, y_test)))

accuracy =  0.6978
